In [1]:
import aocd
import re
import numpy as np
from anytree import Node, RenderTree
import json
import math
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2021, day=18)
data = puzzle.input_data.split("\n")

In [2]:
def string_to_tree(string):
    j = json.loads(string)
    if(isinstance(j[0], int)):
        left = Node(j[0])
    else:
        left = string_to_tree(str(j[0]))
    if(isinstance(j[1], int)):
        right = Node(j[1])
    else:
        right = string_to_tree(str(j[1]))
    ret = Node("")
    ret.children = [left, right]
    return ret

In [3]:
tree_data = [string_to_tree(x) for x in data]

In [4]:
def get_left(n):
    last_node = n
    for node in n.parent.iter_path_reverse():
        if(node.children[1] == last_node):
            return node.children[0].leaves[-1]
        else:
            last_node = node
    else:
        return None

In [5]:
def get_right(n):
    last_node = n
    for node in n.parent.iter_path_reverse():
        if(node.children[0] == last_node):
            return node.children[1].leaves[0]
        else:
            last_node = node
    else:
        return None

In [6]:
def reduce(tree):
    e = [x for x in tree.leaves if x.depth > 4]
    if(len(e) > 0):
        p = e[0].parent
        (l,r) = p.children
        ll = get_left(l)
        if(ll is not None):
            ll.name = int(ll.name) + int(l.name)
        rr = get_right(r)
        if(rr is not None):
            rr.name = int(rr.name) + int(r.name)
        p.name = 0
        p.children = []
        return reduce(tree)
    g = [x for x in tree.leaves if int(x.name) >= 10]
    if(len(g) > 0):
        s = g[0]
        v = int(s.name)
        s.name = ""
        s.children = [Node(math.floor(v/2)), Node(math.ceil(v/2))]
        return reduce(tree)
    return tree
        

In [7]:
def add(n1, n2):
    return reduce(Node("", children=[n1, n2]))

In [8]:
def magnitude(tree):
    if tree.is_leaf:
        return int(tree.name)
    else:
        return 3 * magnitude(tree.children[0]) + 2 * magnitude(tree.children[1])

In [9]:
last = tree_data[0]
for t in tree_data[1:]:
    last = add(last, t)

In [10]:
puzzle.answer_a = magnitude(last)

In [ ]:
from itertools import permutations
p = permutations(data, 2)
max_mag = 0
for t in p:
    # Done because addition is destructive
    max_mag = max(max_mag, magnitude(add(string_to_tree(t[0]), string_to_tree(t[1]))))

In [ ]:
puzzle.answer_b = max_mag